In [1]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
# Carregar os dados preparados
x_treino = np.load('x_treino.npy')  
y_treino = np.load('y_treino.npy')  
z_treino = np.load('z_treino.npy')

x_validacao = np.load('x_validacao.npy')  
y_validacao = np.load('y_validacao.npy')  
z_validacao = np.load('z_validacao.npy')

x_teste = np.load('x_teste.npy')  
y_teste = np.load('y_teste.npy')  
z_teste = np.load('z_teste.npy')

print(np.shape(x_treino))
print(np.shape(x_validacao))
print(np.shape(x_teste))

In [4]:
def plotar_matriz_confusao(respostas, predicoes, rotulos):
    cm = confusion_matrix(respostas, predicoes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=rotulos, yticklabels=rotulos)
    plt.xlabel('Rótulos previstos')
    plt.ylabel('Rótulos verdadeiros')
    plt.title('Matriz de confusão')
    plt.show()

In [ ]:
pca = PCA(n_components=0.95) 
x_treino_pca = pca.fit_transform(x_treino)
x_validacao_pca = pca.transform(x_validacao)
x_teste_pca = pca.transform(x_teste)

print("TREINAMENTO")
print(f"Dimensões originais: {np.shape(x_treino)}")
print(f"Dimensões após PCA: {np.shape(x_treino_pca)}")
print("VALIDAÇÃO")
print(f"Dimensões originais: {np.shape(x_validacao)}")
print(f"Dimensões após PCA: {np.shape(x_validacao_pca)}")
print("TESTE")
print(f"Dimensões originais: {np.shape(x_teste)}")
print(f"Dimensões após PCA: {np.shape(x_teste_pca)}")

# MLP padrão
- hidden_layer_sizes: (100,)
- activation: 'relu'
- solver: 'adam'
- alpha: 0.0001
- batch_size: 'auto'
- learning_rate: "constant"
- learning_rate_init: 0.001
- power_t: 0.5
- max_iter: 200
- shuffle: True
- random_state: None
- tol: 1e-4
- verbose: False
- warm_state: False
- momentum: 0.9
- nesterovs_momentum: True
- early_stopping: False
- validation_fraction: 0.1
- beta_1: 0.9
- beta_2: 0.999
- epsilon: 1e-8
- n_iter_no_change: 10
- max_fun: 15000


In [ ]:
mlp_padrao = MLPClassifier(random_state=42)

In [ ]:
train_sizes, train_scores, test_scores = learning_curve(
    mlp_padrao, x_treino_pca, y_treino, cv=5, scoring='accuracy', train_sizes=np.linspace(0.1, 1.0, 10)
)

train_mean = np.mean(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)

plt.figure(figsize=(8, 6))
plt.plot(train_sizes, train_mean, label='Acurácia no Treinamento', marker='o', color='blue')
plt.plot(train_sizes, test_mean, label='Acurácia no Teste', marker='o', color='orange')
plt.xlabel('Tamanho do Conjunto de Treinamento')
plt.ylabel('Acurácia')
plt.title('Curvas de Aprendizado')
plt.legend()
plt.grid()
plt.show()

In [ ]:
mlp_padrao.fit(x_treino_pca, y_treino)

In [ ]:
predicao_validacao_padrao = mlp_padrao.predict(x_validacao_pca)
treino_acuracia_padrao = accuracy_score(y_validacao, predicao_validacao_padrao)
print("Acurácia (modelo inicial):", treino_acuracia_padrao)
print("Relatório de Classificação (modelo inicial):\n", classification_report(y_validacao, predicao_validacao_padrao))

In [ ]:
predicao_teste_padrao = mlp_padrao.predict(x_teste_pca)
teste_acuracia_padrao = accuracy_score(y_teste, predicao_teste_padrao)
print("Acurácia (modelo inicial):", teste_acuracia_padrao)
print("Relatório de Classificação (modelo inicial):\n", classification_report(y_teste, predicao_teste_padrao))

In [ ]:
# Código para gerar a matriz de confusão.
letras = ["A", "B", "C", "D", "E", "F", "G", "I", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "Y"]
y_prob_teste = mlp_padrao.predict_proba(x_teste_pca)

plotar_matriz_confusao(y_teste, predicao_teste_padrao, letras)

In [ ]:
x_teste_ruidosa = x_teste_pca + np.random.normal(0, 0.1, x_teste_pca.shape)
x_teste_ruidosa = np.clip(x_teste_ruidosa, 0, 1)

y_pred_ruidosa = mlp_padrao.predict(x_teste_ruidosa)
teste_acuracia_padrao = accuracy_score(y_teste, y_pred_ruidosa)

print("Acurácia:", teste_acuracia_padrao)

# MLP otimizado: 
- activation: 'relu'
- hidden_layer_sizes: (100,)
- solver: 'adam'
- max_iter: 100

In [ ]:
param_grid = {
    'hidden_layer_sizes': [(50,), (100,)],
    'max_iter': [50, 100, 200],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
}
grid_search = GridSearchCV(MLPClassifier(random_state=42), param_grid, scoring='accuracy', cv=2, verbose=2)
grid_search.fit(x_treino_pca, y_treino)

In [ ]:
best_mlp = grid_search.best_estimator_

print("Melhores parâmetros encontrados:", grid_search.best_params_)
print("Melhor score encontrado:", grid_search.best_score_)

In [ ]:
mlp_otimizado = MLPClassifier(hidden_layer_sizes=(100,), max_iter=100, random_state=42, solver='adam', activation='relu')

In [ ]:
train_sizes, train_scores, test_scores = learning_curve(
    mlp_otimizado, x_treino_pca, y_treino, cv=5, scoring='accuracy', train_sizes=np.linspace(0.1, 1.0, 10)
)

train_mean = np.mean(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)

plt.figure(figsize=(8, 6))
plt.plot(train_sizes, train_mean, label='Acurácia no Treinamento', marker='o', color='blue')
plt.plot(train_sizes, test_mean, label='Acurácia no Teste', marker='o', color='orange')
plt.xlabel('Tamanho do Conjunto de Treinamento')
plt.ylabel('Acurácia')
plt.title('Curvas de Aprendizado')
plt.legend()
plt.grid()
plt.show()

In [ ]:
mlp_otimizado.fit(x_treino_pca, y_treino)

In [ ]:
predicao_validacao_otimizado = mlp_otimizado.predict(x_validacao_pca)
treino_acuracia_otimizado = accuracy_score(y_validacao, predicao_validacao_otimizado)
print("Acurácia (modelo inicial):", treino_acuracia_otimizado)
print("Relatório de Classificação (modelo inicial):\n", classification_report(y_validacao, predicao_validacao_otimizado))

In [ ]:
predicao_teste_otimizado = mlp_otimizado.predict(x_teste_pca)
teste_acuracia_otimizado = accuracy_score(y_teste, predicao_teste_otimizado)
print("Acurácia (modelo inicial):", teste_acuracia_otimizado)
print("Relatório de Classificação (modelo inicial):\n", classification_report(y_teste, predicao_teste_otimizado))

In [ ]:
# Código para gerar a matriz de confusão.
letras = ["A", "B", "C", "D", "E", "F", "G", "I", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "Y"]
y_prob_teste = mlp_otimizado.predict_proba(x_teste_pca)

plotar_matriz_confusao(y_teste, predicao_teste_otimizado, letras)

In [ ]:
x_teste_ruidosa = x_teste_pca + np.random.normal(0, 0.1, x_teste_pca.shape)
x_teste_ruidosa = np.clip(x_teste_ruidosa, 0, 1)

y_pred_ruidosa = mlp_otimizado.predict(x_teste_ruidosa)
teste_acuracia_padrao = accuracy_score(y_teste, y_pred_ruidosa)

print("Acurácia:", teste_acuracia_padrao)